In [38]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import pandas as pd


In [39]:
!pip install html5lib

In [40]:
# # 우리가 찾은 url이 아닌 그냥 한화은행 환율정보 바로 들어갔을 때 url임
# # 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
# options = webdriver.ChromeOptions()
# options.add_experimental_option("excludeSwitches", ['enable-logging'])
# options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
# options.add_argument('lang=ko_KR')

# # 크롬 웹브라우저 드라이버 자동 다운로드
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver.set_window_size(1920, 1080) # 웹브라우저 해상도 조절
# driver.get("https://www.kebhana.com/cont/mall/mall15/mall1501/index.jsp")
# page_html = driver.page_source
# df = pd.read_html(page_html)

In [41]:
# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-logging'])
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
options.add_argument('lang=ko_KR')

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹브라우저 해상도 조절
driver.get("https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")

In [42]:
page_html = driver.page_source

In [43]:
page_html

'<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko"><head>\n<title>하나은행 개인뱅킹</title>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta http-equiv="Cache-control" content="No-cache">\n<meta http-equiv="Expires" content="0">\n<meta http-equiv="Pragma" content="No-cache">\n<meta name="Description" content="">\n<meta name="Keywords" content="">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n<link rel="shortcut icon" href="/favicon.ico">\n<script async="" charset="utf-8" src="https://logcol.kebhana.com:8443/static/96580/ntm.js?ver=1722470400000"></script><script async="" charset="utf-8" src="https://logcol.kebhana.com:8443/static/96580/install.js?ver=1722470400000"></script><script type="text/javascript">\nvar TNK_SR = \'67e0d1db4042503730933c41a122b031\';\nvar _W_L_M_ = \'P\';\nvar DOMAIN_HOST_NM = \'pbk.prdhostNm\';\nvar isTransKey       = false;  // 마우스입력기 사용여부\n\n \nwindow._P1_CAUTH_LOG = \'SZSb1AqKBIUmIJWchl3gfg==:39713bea-7ae0-4398-

In [44]:
pd.read_html(page_html)[0]

ValueError: No tables found

In [ ]:
df = pd.read_html(page_html)[1]
df

In [ ]:
search_box = driver.find_element(By.CSS_SELECTOR, "#tmpInqStrDt") # 얘는 id이므로 유일함 무조건 저거 하나만 쳐도 나옴
search_box.clear()
search_box.send_keys("2024-07-31"+Keys.ENTER)

In [ ]:
search_button = driver.find_element(By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")
search_button.click()

In [ ]:
# 얘는 selenium에서 필요 없는 애임
#table = soup.select_one("#searchContentDiv > div.printdiv > table")
#table_html = str(table)

In [ ]:
from sqlalchemy import create_engine, text
import pymysql
from exdbinfo import db, dbtype, id, pw, host, database
# pymysql을 MySQLdb로 설치
pymysql.install_as_MySQLdb()


# 데이터베이스 엔진 생성
engine = create_engine("%s+%s://%s:%s@%s" % (db, dbtype, id, pw, host))
conn = engine.connect()

# 데이터베이스가 없으면 생성
conn.execute(text('CREATE DATABASE IF NOT EXISTS korea_exchange_rate'))
conn.close()

In [ ]:
# DB에 접속해서 저장하는 함수
def bank_info_to_db(df):
    from sqlalchemy import create_engine
    import pymysql
    from exdbinfo import db, dbtype, id, pw, host, database
    from datetime import date
    pymysql.install_as_MySQLdb()

    today = str(date.today()).replace("-","_")
    print(db, dbtype, id, pw, host, database)
    # bank_info 데이터 베이스 연결
    engine = create_engine("%s+%s://%s:%s@%s/%s" % (db, dbtype, id, pw, host, database))
    conn = engine.connect()
    df.to_sql(f"bank_info", con=conn, if_exists='append', index=False)
    conn.close()
    
    return print(f"{'저장완료':<30s}", end="\r")
bank_info_to_db(df)

# 매일 수집하기
* 강사님

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
import dbio
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [8]:
# 하루 수집하기
today = str(date.today())

## 위에 있는거 합쳐서 한번에
# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Headless 모드 (창이 안뜨게 하는거임 - 12시간 이렇게 창을 계속 띄워놓으면 작업하는데 불편하기 때문)
# options.add_experimental_option("excludeSwitches", ['enable-logging'])
# options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
# options.add_argument('lang=ko_KR')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1920, 1080) # 웹브라우저 해상도 조절
driver.get("https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")

wait = WebDriverWait(driver, 10) # 웹요소가 나타날 때까지 최대 10초 대기

# 날짜 변경
# 얘는 id이므로 유일함 무조건 저거 하나만 쳐도 나옴
# #tmpInqStrDt이 요소가 나오면 바로 실행됨 -> 안나오면 10초동안 대기
search_box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqStrDt")))
search_box.clear()
search_box.send_keys(today+Keys.ENTER)

# 조회하기 버튼
search_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
search_button.click()

# 환율정보 테이블이 뜰 때까지 기다림
#searchContentDiv > div.printdiv > table
exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table")))

# html 소스를 읽어서 데이터프레임으로 만들고 DB저장
page_html = driver.page_source
df = pd.read_html(page_html)[1]
df['date'] = today
cols = ('통화', '현찰_사실때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', '송금_보낼때', '송금_받을때', 'T/C_살때', '외화_수표_팔때', '매매기준율', '환가_료율', '미화환산율', 'date')
df.columns = cols
sorted_cols = ('date','통화', '현찰_사실때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', '송금_보낼때', '송금_받을때', 'T/C_살때', '외화_수표_팔때', '매매기준율', '환가_료율', '미화환산율')
df = df[[*sorted_cols]]



# db에 저장
dbio.exi_to_db("exchange_rate", today, df)


In [11]:
df['date'] = date

In [ ]:
df

In [17]:
# 컬럼만 뽑을 때
df.columns

MultiIndex([(       '통화',        '통화',        '통화'),
            (       '현찰',      '사실 때',        '환율'),
            (       '현찰',      '사실 때',    'Spread'),
            (       '현찰',      '파실 때',        '환율'),
            (       '현찰',      '파실 때',    'Spread'),
            (       '송금',      '보낼 때',      '보낼 때'),
            (       '송금',      '받을 때',      '받을 때'),
            (  'T/C 사실때',   'T/C 사실때',   'T/C 사실때'),
            ('외화 수표 파실때', '외화 수표 파실때', '외화 수표 파실때'),
            (   '매매 기준율',    '매매 기준율',    '매매 기준율'),
            (    '환가 료율',     '환가 료율',     '환가 료율'),
            (   '미화 환산율',    '미화 환산율',    '미화 환산율'),
            (     'date',          '',          '')],
           )

In [24]:
cols = ('통화', '현찰_사실때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', '송금_보낼때', '송금_받을때', 'T/C_살때', '외화_수표_팔때', '매매기준율', '환가_료율', '미화환산율', 'date')
# sorted_cols = ('date','통화', '현찰_사실때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', '송금_보낼때', '송금_받을때', 'T/C_살때', '외화_수표_팔때', '매매기준율', '환가_료율', '미화환산율')

# 1995년 1월 3일 ~ 2024년 7월 31일까지 평일 환율 정보 수집

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
import dbio
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

## 날짜 리스트 생성하기

In [25]:
import pandas as pd

# 시작 날짜와 끝 날짜 설정
start_date = '1995-01-03'
end_date = '2024-07-31'

# 날짜 범위 생성
date_range = pd.date_range(start=start_date, end=end_date, freq='B')

# 리스트로 변환
date_list = date_range.strftime('%Y-%m-%d').tolist()

# print(date_list)
print(type(date_list))
print(len(date_list))

<class 'list'>
7717


In [13]:
str(date_list[0])

'1995-01-03'

In [15]:
for weekday in date_list[:5]:
    print(str(weekday))

1995-01-03
1995-01-04
1995-01-05
1995-01-06
1995-01-09


In [ ]:
for weekday in date_list:
    today = (str(weekday))
        
    
    ## 위에 있는거 합쳐서 한번에
    # 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # Headless 모드 (창이 안뜨게 하는거임 - 12시간 이렇게 창을 계속 띄워놓으면 작업하는데 불편하기 때문)
    # options.add_experimental_option("excludeSwitches", ['enable-logging'])
    # options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
    # options.add_argument('lang=ko_KR')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    # 크롬 웹브라우저 드라이버 자동 다운로드
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1920, 1080) # 웹브라우저 해상도 조절
    driver.get("https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")
    time.sleep(3)
    wait = WebDriverWait(driver, 30) # 웹요소가 나타날 때까지 최대 10초 대기
    
    # 날짜 변경
    # 얘는 id이므로 유일함 무조건 저거 하나만 쳐도 나옴
    # #tmpInqStrDt이 요소가 나오면 바로 실행됨 -> 안나오면 10초동안 대기
    search_box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqStrDt")))
    search_box.clear()
    search_box.send_keys(today+Keys.ENTER)
    
    # 조회하기 버튼
    search_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
    search_button.click()
    time.sleep(5)
    
    # 환율정보 테이블이 뜰 때까지 기다림
    #searchContentDiv > div.printdiv > table
    exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table")))
    
    # html 소스를 읽어서 데이터프레임으로 만들고 DB저장
    page_html = driver.page_source
    df = pd.read_html(page_html)[1]
    df['date'] = today
    cols = ('통화', '현찰_사실때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', '송금_보낼때', '송금_받을때', 'T/C_살때', '외화_수표_팔때', '매매기준율', '환가_료율', '미화환산율', 'date')
    df.columns = cols
    sorted_cols = ('date','통화', '현찰_사실때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', '송금_보낼때', '송금_받을때', 'T/C_살때', '외화_수표_팔때', '매매기준율', '환가_료율', '미화환산율')
    df = df[[*sorted_cols]]
    
    
    
    # db에 저장
    dbio.exi_to_db("exchange_rate_1995_2024", today, df)
